In [ ]:
import copy
from itertools import cycle
from typing import Optional, Union

import os
from datetime import datetime, timedelta
from time import sleep
# import plotly.express as px
# import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from glob import glob
import requests_cache
import numpy as np
from sympy.physics.units import years

from data_collection_modules import DataEnergySMARD
from data_collection_modules import locations, OpenMeteo

# Load All Dataframes

In [ ]:
data_dir = '../database/'
target = 'wind_offshore'
# df = pd.read_parquet(data_dir + 'latest.parquet')
df_smard = pd.read_parquet(data_dir + 'smard/' + 'history.parquet')
df_om = pd.read_parquet(data_dir + 'openmeteo/' + 'history.parquet')
df_om_f = pd.read_parquet(data_dir + 'openmeteo/' + 'forecast.parquet')
df_es = pd.read_parquet(data_dir + 'epexspot/' + 'history.parquet')

print(f"SMARD data shapes hist={df_smard.shape} start={df_smard.index[0]} end={df_smard.index[-1]}")
print(f"Openmeteo data shapes hist={df_om.shape} start={df_om.index[0]} end={df_om.index[-1]}")
print(f"Openmeteo data shapes forecast={df_om_f.shape} start={df_om_f.index[0]} end={df_om_f.index[-1]}")
print(f"EPEXSPOT data shapes hist={df_es.shape} start={df_es.index[0]} end={df_es.index[-1]}")

print(f"Target={target} Nans={df_smard[target].isna().sum().sum()}")
# set how to split the dataset
cutoff = df_om_f.index[0]
if cutoff == cutoff.normalize():
    print(f"The cutoff timestamp corresponds to the beginning of the day {cutoff.normalize()}")
print(f"Dataset is split into ({len(df_om[:cutoff])}) before and "
      f"({len(df_om_f[cutoff:])}) ({int(len(df_om_f[cutoff:])/24)} days) after {cutoff}.")
print(df_smard.columns.to_list())

In [ ]:
# df_smard = pd.read_parquet(data_dir + 'smard/' + 'history.parquet')
# df_smard_ = df_smard[:df_smard.index[-1]-timedelta(days=180)]
# df_smard_.to_parquet(data_dir + 'smard/' + 'history.parquet')

# Feature Engineering For Weather Data (Historic and Forecast)

In [ ]:
# create combined dataframe
cutoff = df_om_f.index[0]
df_om = df_om.combine_first(df_om_f)
if df_om.isna().any().any():
    print("ERROR! Nans in the dataframe")
def preprocess_openmeteo_for_offshore_wind_OLD(df, location_suffix="_hsee"):
    """
    Preprocesses weather data for forecasting offshore wind energy generation.
    """
    # 1. Filter for relevant location
    cols_to_keep = [c for c in df.columns if c.endswith(location_suffix)]
    df = df[cols_to_keep].copy()

    # 2. Key variable columns
    wind_speed_10m_col = f"wind_speed_10m{location_suffix}"
    wind_speed_100m_col = f"wind_speed_100m{location_suffix}"
    wind_dir_10m_col = f"wind_direction_10m{location_suffix}"
    wind_dir_100m_col = f"wind_direction_100m{location_suffix}"
    temp_col = f"temperature_2m{location_suffix}"
    press_col = f"surface_pressure{location_suffix}"
    rh_col = f"relative_humidity_2m{location_suffix}"

    # 3. Wind direction (cyclic encoding)
    if wind_dir_10m_col in df.columns:
        df["wind_dir_10m_sin"] = np.sin(np.deg2rad(df[wind_dir_10m_col]))
        df["wind_dir_10m_cos"] = np.cos(np.deg2rad(df[wind_dir_10m_col]))
        df.drop(columns=[wind_dir_10m_col], inplace=True, errors="ignore")
    if wind_dir_100m_col in df.columns:
        df["wind_dir_100m_sin"] = np.sin(np.deg2rad(df[wind_dir_100m_col]))
        df["wind_dir_100m_cos"] = np.cos(np.deg2rad(df[wind_dir_100m_col]))
        df.drop(columns=[wind_dir_100m_col], inplace=True, errors="ignore")

    # 4. Wind speed transformations
    if wind_speed_10m_col in df.columns and wind_speed_100m_col in df.columns:
        # Wind Shear
        df["wind_shear"] = np.log(df[wind_speed_100m_col] / df[wind_speed_10m_col]) / np.log(100 / 10)
        # Wind energy potential (v^3 for both heights)
        df["wind_power_potential_10m"] = df[wind_speed_10m_col] ** 3
        df["wind_power_potential_100m"] = df[wind_speed_100m_col] ** 3

        # Lag and rolling features for wind speed at 100m
        for lag in [1, 6, 12, 24]:
            df[f"{wind_speed_100m_col}_lag{lag}"] = df[wind_speed_100m_col].shift(lag)
        df[f"{wind_speed_100m_col}_roll6"] = df[wind_speed_100m_col].rolling(window=6, min_periods=1).mean()
        df[f"{wind_speed_100m_col}_roll24"] = df[wind_speed_100m_col].rolling(window=24, min_periods=1).mean()

    # 5. Turbulence Intensity
    if wind_speed_100m_col in df.columns:
        rolling_std = df[wind_speed_100m_col].rolling(window=3, min_periods=1).std()
        rolling_mean = df[wind_speed_100m_col].rolling(window=3, min_periods=1).mean()
        df["turbulence_intensity"] = rolling_std / rolling_mean

    # 6. Wind Ramp Events
    if wind_speed_100m_col in df.columns:
        df["wind_ramp"] = df[wind_speed_100m_col].diff(periods=1)  # Change in wind speed over 1 timestep

    # 7. Moisture Index
    if temp_col in df.columns and rh_col in df.columns:
        df["moisture_index"] = df[temp_col] * df[rh_col]

    # 8. Thermal Stability Index
    if temp_col in df.columns and press_col in df.columns:
        df["thermal_stability_index"] = (df[temp_col] + 273.15) * (1000 / df[press_col]) ** 0.286

    # 9. Drop irrelevant features
    drop_vars = ["precipitation", "cloud_cover", "shortwave_radiation"]
    drop_cols = [f"{var}{location_suffix}" for var in drop_vars if f"{var}{location_suffix}" in df.columns]
    df.drop(columns=drop_cols, inplace=True, errors="ignore")

    return df
def preprocess_openmeteo_for_offshore_wind(df, location_suffix="_hsee")->pd.DataFrame:
    """
    Preprocesses weather data for forecasting offshore wind energy generation.
    Focuses on critical physical features and includes turbulence_intensity, wind_ramp, and wind_shear.
    """

    # 1. Filter for the offshore wind farm location only
    cols_to_keep = [c for c in df.columns if c.endswith(location_suffix)]
    df = df[cols_to_keep].copy()

    # 2. Define key variable columns
    wind_speed_10m_col = f"wind_speed_10m{location_suffix}"
    wind_speed_100m_col = f"wind_speed_100m{location_suffix}"
    wind_dir_100m_col = f"wind_direction_100m{location_suffix}"
    temp_col = f"temperature_2m{location_suffix}"
    press_col = f"surface_pressure{location_suffix}"

    # 3. Compute Air Density (ρ)
    if temp_col in df.columns and press_col in df.columns:
        temp_K = df[temp_col] + 273.15
        R_specific = 287.05  # J/(kg·K) for dry air
        # convert pressure from hPa to Pa
        df["air_density"] = np.array( (df[press_col] * 100.) / (R_specific * temp_K) )

    # 4. Compute Wind Power Density (if wind_speed_100m and air_density are available)
    if wind_speed_100m_col in df.columns and "air_density" in df.columns:
        df["wind_power_density"] = np.array( 0.5 * df["air_density"] * (df[wind_speed_100m_col] ** 3) )

    # 5. Encode Wind Direction (Cyclic)
    if wind_dir_100m_col in df.columns:
        df["wind_dir_sin"] = np.sin(np.deg2rad(df[wind_dir_100m_col]))
        df["wind_dir_cos"] = np.cos(np.deg2rad(df[wind_dir_100m_col]))
        df.drop(columns=[wind_dir_100m_col], inplace=True)

    # 6. Wind Shear (Requires both 10m and 100m wind speeds)
    if wind_speed_10m_col in df.columns and wind_speed_100m_col in df.columns:
        with np.errstate(divide='ignore', invalid='ignore'):
            df["wind_shear"] = np.log(df[wind_speed_100m_col] / df[wind_speed_10m_col]) / np.log(100/10)
        # Replace infinities or NaNs if they occur
        df["wind_shear"].replace([np.inf, -np.inf], np.nan, inplace=True)

    # 7. Turbulence Intensity (using a short rolling window on 100m wind speed)
    if wind_speed_100m_col in df.columns:
        rolling_std = df[wind_speed_100m_col].rolling(window=3, min_periods=1).std()
        rolling_mean = df[wind_speed_100m_col].rolling(window=3, min_periods=1).mean()
        df["turbulence_intensity"] = np.array( rolling_std / rolling_mean )

    # 8. Wind Ramp (difference in 100m wind speed over 1 timestep)
    if wind_speed_100m_col in df.columns:
        df["wind_ramp"] = df[wind_speed_100m_col].diff(1)

    # 9. Lag Features for Wind Speed at 100m
    if wind_speed_100m_col in df.columns:
        for lag in [1, 6, 12, 24]:
            df[f"wind_speed_lag_{lag}"] = df[wind_speed_100m_col].shift(lag)

    # 11. Drop Irrelevant Columns
    # Decide which columns to drop. For model simplicity, consider dropping raw weather inputs
    # that have been transformed into more physical parameters.
    # However, keep wind speeds if you think they add value.
    # For now, we keep the wind speeds since other derived features depend on them.
    drop_vars = [
        temp_col, press_col, "air_density",
        f"precipitation{location_suffix}",
        f"cloud_cover{location_suffix}",
        f"shortwave_radiation{location_suffix}",
        f"relative_humidity_2m{location_suffix}",
        f"wind_direction_10m{location_suffix}",
        f"wind_gusts_10m{location_suffix}"
    ]
    drop_cols = [c for c in drop_vars if c in df.columns]
    df.drop(columns=drop_cols, inplace=True, errors="ignore")

    # Handle missing values introduced by lagging and other computations
    # df.dropna(inplace=True)

    return df
df_om_prep = preprocess_openmeteo_for_offshore_wind(df=df_om, location_suffix="_hsee")
print(df_om_prep.shape, df_om_prep.columns, df_om_prep.isna().sum())
df_om_prep.dropna(inplace=True)

In [ ]:
def visualize_weather_smard(df_om_prep_, df_smard_, target=target):
    # df_om_prep_ = df_om_prep[:cutoff].tail(30*24)
    # df_smard_ = df_smard.tail(30*24)
    fig,axes = plt.subplots(ncols=1,nrows=6,figsize=(10,10), sharex='all')
    ax = axes[0]
    ax.plot(df_smard_.index, df_smard_[target])
    ax = axes[1]
    ax.plot(df_om_prep_.index, df_om_prep_["wind_speed_100m_hsee"])
    ax = axes[2]
    ax.plot(df_om_prep_.index, df_om_prep_["wind_shear"])
    ax = axes[3]
    ax.plot(df_om_prep_.index, df_om_prep_["turbulence_intensity"])
    ax = axes[4]
    ax.plot(df_om_prep_.index, df_om_prep_["wind_ramp"])
    ax = axes[5]
    ax.plot(df_om_prep_.index, df_om_prep_["wind_power_density"])
    plt.show()
visualize_weather_smard(df_om_prep[:cutoff].tail(90*24), df_smard.tail(90*24))

# Split Data Into Hist and Forecast and Combine With SMARD Data

In [ ]:
# Visualize the split between data
if cutoff == cutoff.normalize():
    print("The cutoff timestamp corresponds to the beginning of the day.")
print(cutoff)
df_om_prep[:cutoff-timedelta(hours=1)].tail(30*24)['wind_speed_10m_hsee'].plot()
df_om_prep[cutoff:].head(24)['wind_speed_10m_hsee'].plot()
plt.axvline(cutoff)

In [ ]:
def handle_nans_with_interpolation(df: pd.DataFrame, name: str) -> pd.DataFrame:
    """
    Checks each column of the DataFrame for NaNs. If a column has more than 3 consecutive NaNs,
    it raises a ValueError. Otherwise, fills the NaNs using bi-directional interpolation.
    """

    df_copy = df.copy()

    def check_consecutive_nans(series: pd.Series):
        # Identify consecutive NaNs by grouping non-NaN segments and counting consecutive NaNs
        consecutive_nans = (series.isna().astype(int)
                            .groupby((~series.isna()).cumsum())
                            .cumsum())
        if consecutive_nans.max() > 3:
            raise ValueError(f"Column '{series.name}' in {name} contains more than 3 consecutive NaNs.")

    # Check all columns for consecutive NaNs first
    for col in df_copy.columns:
        check_consecutive_nans(df_copy[col])

    # Interpolate all columns at once after confirming they're valid
    df_copy = df_copy.interpolate(method='linear', limit_direction='both', axis=0)

    return df_copy
def fix_broken_periodicity_with_interpolation(df: pd.DataFrame, name: str) -> pd.DataFrame:
    """
    Fixes broken hourly periodicity by adding missing timestamps if fewer than 3 consecutive are missing.
    Raises an error if more than 3 consecutive timestamps are missing.
    Missing values are filled using time-based interpolation.
    """

    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError(f"The DataFrame {name} must have a datetime index.")

    expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
    missing_timestamps = expected_index.difference(df.index)

    if missing_timestamps.empty:
        print(f"The DataFrame {name} is already hourly with no missing segments.")
        return df

    # Convert to a Series to check consecutive missing timestamps
    missing_series = pd.Series(missing_timestamps)
    groups = (missing_series.diff() != pd.Timedelta(hours=1)).cumsum()

    # Check if any group has more than 3 missing points
    group_counts = groups.value_counts()
    if (group_counts > 3).any():
        bad_group = group_counts[group_counts > 3].index[0]
        raise ValueError(f"More than 3 consecutive missing timestamps detected: "
                         f"{missing_series[groups == bad_group].values} in {name}")

    # Reindex and interpolate
    fixed_df = df.reindex(expected_index)
    fixed_df = fixed_df.interpolate(method='time')

    print(f"Added and interpolated {len(missing_timestamps)} missing timestamps in {name}.")

    return fixed_df
def validate_dataframe(df: pd.DataFrame, name: str = '') -> pd.DataFrame:
    """Check for NaNs, missing values, and periodicity in a time-series DataFrame."""

    # Check for NaNs
    if df.isnull().any().any():
        print(f"ERROR! {name} DataFrame contains NaN values.")
        df = handle_nans_with_interpolation(df, name)

    # Check if index is sorted in ascending order
    if not df.index.is_monotonic_increasing:
        print(f"ERROR! {name} The index is not in ascending order.")
        raise ValueError("Data is not in ascending order.")

    # Check for hourly frequency with no missing segments
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
    if not full_range.equals(df.index):
        print(f"ERROR! {name} The data is not hourly or has missing segments.")
        df = fix_broken_periodicity_with_interpolation(df, name)

    return df

horizon = 7 * 24
# merger with SMRD target column
target = 'wind_offshore'
df_om_prep.dropna(inplace=True, how='any')
df_hist = pd.merge(
    df_om_prep[:cutoff-timedelta(hours=1)],
    df_smard[:cutoff-timedelta(hours=1)][target],
    left_index=True, right_index=True, how="inner"
)
df_forecast = df_om_prep[cutoff : cutoff+timedelta(hours=horizon - 1)]
df_hist = validate_dataframe(df_hist, 'df_hist')
df_forecast = validate_dataframe(df_forecast, 'df_forecast')
df_hist = df_hist[df_hist.index[-1]-pd.Timedelta(hours = 100 * horizon - 1):]
print(f"Features {len(df_hist.columns)-1} hist.shape={df_hist.shape} ({int(len(df_hist)/horizon)}) forecast.shape={df_forecast.shape}")
print(f"Hist: from {df_hist.index[0]} to {df_hist.index[-1]} ({len(df_hist)/horizon})")
print(f"Fore: from {df_forecast.index[0]} to {df_forecast.index[-1]} ({len(df_forecast)/horizon})")

outdir = "../tmp_database/"
df_hist.to_parquet(f"{outdir}history.parquet")
df_forecast.to_parquet(f"{outdir}forecast.parquet")

In [ ]:
df_hist[target].tail(512).plot()

# Visualize

In [ ]:
from forecasting_modules import compute_timeseries_split_cutoffs, compute_error_metrics

cutoffs = compute_timeseries_split_cutoffs(
    df_hist.index,
    horizon=len(df_forecast.index),
    delta=len(df_forecast.index),
    folds=5,
    min_train_size=30*24
)
smard_res = []
results = {}
smard_metrics = []
for i, cutoff in enumerate(cutoffs):
    mask = (df_hist.index >= cutoff) & (df_hist.index < cutoff + pd.Timedelta(hours=len(df_forecast.index)))
    mask_ = (df_smard.index >= cutoff) & (df_smard.index < cutoff + pd.Timedelta(hours=len(df_forecast.index)))
    actual = df_hist[target][mask]
    predicted = df_smard[f"{target}_forecasted"][mask_]
    # print(f"{predicted.index[0]}")
    # print(f"\t{predicted.index[-1]}")
    df = pd.DataFrame({
        f'{target}_actual':actual.values,
        f'{target}_fitted': predicted.values,
        f'{target}_lower': np.zeros_like(actual.values),
        f'{target}_upper': np.zeros_like(actual.values)
    }, index=actual.index)
    smard_res.append(copy.deepcopy(df))
    smard_metrics.append( compute_error_metrics(target, df) )


ave_metrics = {
    key: np.mean( [smard_metrics[i][key] for i in range(len((smard_metrics)))] ) for key in list(smard_metrics[0].keys())
}
smard_metrics.append(ave_metrics)


In [ ]:
working_dir = f'../forecasting_modules/output/'
from forecasting_modules import ForecastingTaskSingleTarget
# Load our past and current forecasts
forecast_res:dict = ForecastingTaskSingleTarget._load_trained_model(
    target=target,
    model_label="ensemble[XGBoost](XGBoost,ElasticNet)",#'ensemble[XGBoost](XGBoost,ElasticNet)',
    working_dir=working_dir,
    train_forecast='forecast',
    verbose=True
)

n = 4 # number of windows to show


smard_forecast:pd.DataFrame = copy.deepcopy(smard_res[-1])
smard_forecast[:] = 0 # unknown, as SMARD does not provide week-ahead forecast from now
tasks = [
    {'model':'SMARD','n':n,'name':'TSO Day-ahead forecast','lw':1.0,'color':'blue','ci_alpha':0.0,
     'results':smard_res[-n:],'metrics':smard_metrics[-n-1:],'forecast':None}, #
    {'model':'XGBoost','n':n,'name':'Our Week-ahead Forecast','lw':1.0,'color':'red','ci_alpha':0.0,
     'results':forecast_res['results'][-n:],'metrics':forecast_res['metrics'][-n-1:],'forecast':None} # forecast_res['forecast']
]
# plot_time_series_with_residuals(
#     tasks=tasks,
#     target=target,
#     ylabel='Off-shore Wind Generation',
# )fig,a
fig,ax = plt.subplots()
smard_res[-1][f'{target}_actual'].plot(ax=ax, color='blue')
forecast_res['results'][-1][f'{target}_actual'].plot(ax=ax, color='red')

In [ ]:
def plot_time_series_with_residuals(tasks: list[dict], target: str = 'total_grid_load', ylabel: str = '',**kwargs):
    '''

    Plots forecasts split into adjacent windows. Each window can show several forecasts performed with
    different methods listed in 'tasks', where each entry is a dictionary with 'results', 'metrics' and 'forecast'.
    Last panel shows the latest forecast (as given in 'forecast' in tasks) and last metrics from 'metrics' in tasks.
    Bottom panels show residuals between actual target variable and forecasted.

    :param tasks: list[dict] where each dict represents a model's forecasting results that consist of
    - tasks[0]['results']:list[pd.DataFrame] list of forecasts for past forecasting winows where each dataframe has:
    f'{target}_actual', f'{target}_fitted', f'{target}_lower', 'f'{target}_upper'
    - tasks[0]['metrics]:list[dict] list of performance metrics for each forecasted window (RMSE,sMAPE...) where the last
    element in the list contains the average metrics
    tasks[0]['forecast']:pd.Dataframe -- same as dataframes in 'results' but with the current latest forecast
    :param target: str target name
    :param label: y-label for the top panels
    :param kwargs: additional arguments for plotting
    :return: None
    '''

    plot_residuals = False
    if 'residuals' in kwargs:
        plot_residuals = kwargs['residuals']

    legends_per_panel = False
    if 'legends_per_panel' in kwargs:
        legends_per_panel = kwargs['legends_per_panel']

    label_errs = True
    if 'label_errs' in kwargs:
        label_errs = kwargs['label_errs']

    # Determine the maximum number of results across tasks
    plot_forecast = False
    for task in tasks:
        if task['forecast']:
            plot_forecast = True

    max_n_results = max(len(task['results']) for task in tasks)
    if plot_forecast: n_cols = max_n_results + 1  # Plus one for 'forecast'
    else: n_cols = max_n_results


    if not 'drawstyle' in kwargs: drawstyle='default'
    else: drawstyle=kwargs['drawstyle']

    # Create figure and axes
    fig, axes = plt.subplots(
        nrows = 2 if plot_residuals else 1, ncols=n_cols,
        figsize=kwargs['figsize'] if 'figsize' in kwargs else (n_cols * 5, 8),
        gridspec_kw={
            'height_ratios': [3, 1] if plot_residuals else [1],
            'hspace': 0.02, 'wspace': 0.01
        },
        sharex='col', sharey='row'
    )

    # Define column names
    actual_col = f'{target}_actual'
    fitted_col = f'{target}_fitted'
    lower_col = f'{target}_lower'
    upper_col = f'{target}_upper'

    # For each column

    for i in range(n_cols):
        ax_top = axes[0, i] if plot_residuals else axes[i]
        ax_bottom = axes[1, i] if plot_residuals else None

        # Flag to check if 'Actual' data has been plotted
        actual_plotted = False

        # For each task
        for task in tasks:
            name = task['name']
            color = task['color']
            ci_alpha=task['ci_alpha']
            lw=task['lw']
            # Determine if the task has data for this column
            if i < len(task['results']):
                df = task['results'][i]
                errs = task['metrics'][i]
            elif i == max_n_results:
                df = task['forecast']
                errs = task['metrics'][i]
            else:
                continue  # Skip plotting for this task in this column

            # Plot 'Actual' data once per subplot
            if not actual_plotted : #and i != n_cols-1:
                ax_top.plot(df.index, df[actual_col], label='Actual', color='black', drawstyle=drawstyle, lw=1.5)
                actual_plotted = True

            # Plot fitted data
            if label_errs and (errs is not None and i != n_cols-1):
                label = name + ' '  fr"RMSE={errs['rmse']:.1f}" + fr" sMAPE={errs['smape']:.1f}"
            elif label_errs:
                label = name + ' ' fr"$\langle$RMSE$\rangle$={errs['rmse']:.1f}" \
                         + fr" $\langle$sMAPE$\rangle$={errs['smape']:.1f}"
            else:
                label = name
            ax_top.plot(df.index, df[fitted_col], label=label, color=color, drawstyle=drawstyle, lw=lw)

            # Plot confidence intervals
            if ci_alpha > 0.:
                ax_top.fill_between(df.index, df[lower_col], df[upper_col], color=color, alpha=ci_alpha)

            # Plot residuals in the bottom panel
            residuals = (df[actual_col] - df[fitted_col]) #/ df[actual_col]
            if ax_bottom: ax_bottom.plot(df.index, residuals, label=name, color=color, drawstyle=drawstyle, lw=lw)

            # limit plots
            ax_top.set_xlim(df.index.min(),df.index.max())
            if ax_bottom: ax_bottom.set_xlim(df.index.min(),df.index.max())
            if 'ylim0' in kwargs: ax_top.set_ylim(kwargs['ylim0'][0], kwargs['ylim0'][1])
            if 'ylim1' in kwargs and ax_bottom: ax_bottom.set_ylim(kwargs['ylim1'][0], kwargs['ylim1'][1])

        # print(f"N={len(df.index)} idx0={df.index[0].isoformat()}")
        # Add a horizontal line at y=0 in residuals plot
        if ax_bottom: ax_bottom.axhline(0, color='gray', linestyle='--', linewidth=1)

        # Set titles and labels
        if i < max_n_results:
            ax_top.set_title(f'Week {df.index[-1].isocalendar().week} of 2024', fontsize=14, weight='bold')
        else:
            ax_top.set_title('Current Forecast', fontsize=14, weight='bold')

        if ylabel and i == 0:
            ax_top.set_ylabel(ylabel)
            if ax_bottom: ax_bottom.set_ylabel('Residual / Actual')

        # legend in the empty area in residual plots
        if i == n_cols - 1 and ax_bottom:
            ax_bottom.legend(loc='upper left', ncol=1, fontsize=10)

        if legends_per_panel or i == n_cols - 1:
            ax_top.legend(loc='upper left', ncol=1, fontsize=10)

        for ax in [ax_top, ax_bottom] if plot_residuals else [ax_top]:
            ax.grid(True, linestyle='-', alpha=0.4)
            ax.tick_params(axis='x', direction='in', bottom=True)
            ax.tick_params(axis='y', which='both', direction='in', left=True, right=True)
            # Set border lines transparent by setting the edge color and alpha
            ax.spines['top'].set_edgecolor((1, 1, 1, 0))  # Transparent top border
            ax.spines['right'].set_edgecolor((1, 1, 1, 0))  # Transparent right border
            ax.spines['left'].set_edgecolor((1, 1, 1, 0))  # Transparent left border
            ax.spines['bottom'].set_edgecolor((1, 1, 1, 0))  # Transparent bottom border

            # Make x and y ticks transparent
            ax.tick_params(axis='x', color=(1, 1, 1, 0))  # Transparent x ticks
            ax.tick_params(axis='y', color=(1, 1, 1, 0))  # Transparent y ticks

            # Make x and y tick labels transparent
            # for tick_label in ax.get_xticklabels():
            #     tick_label.set_color((1, 1, 1, 0))  # Transparent x tick labels
            # for tick_label in ax.get_yticklabels():
            #     tick_label.set_color((1, 1, 1, 0))  # Transparent y tick labels


        # Improve x-axis formatting
        # ax_bottom.set_xlabel(f'Date (month-day for $2024$)', fontsize=12)
        if ax_bottom:
            ax_bottom.xaxis.set_major_locator(mdates.DayLocator())
            # ax_bottom.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
            ax_bottom.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))  # Format as "Dec 15"
        else:
            ax_top.xaxis.set_major_locator(mdates.DayLocator())
            # ax_bottom.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
            ax_top.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))  # Format as "Dec 15"
        fig.autofmt_xdate(rotation=45)


    model_names = "".join(task["name"]+'_' for task in tasks)
    plt.savefig(f'{target}_{model_names}.png', bbox_inches='tight')
    plt.show()

# Usage: Just call this function similarly to your existing plotting call
plot_time_series_with_residuals(tasks, target=target, ylabel='Off-shore Wind Generation', residuals = False, legends_per_panel=False, label_errs=False, figsize=(12,4))

In [ ]:
def plot_broken_periodicity(df, target):
    """
    Plots the DataFrame and highlights where periodicity is broken.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame with a datetime index.
    """
    # Ensure the index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame must have a datetime index.")

    # Generate the expected hourly index
    expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')

    # Find missing or unexpected timestamps
    missing_timestamps = expected_index.difference(df.index)
    extra_timestamps = df.index.difference(expected_index)

    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.plot(df[target], marker='.', label='Data')

    # Highlight missing timestamps
    for ts in missing_timestamps:
        plt.axvline(ts, color='red', linestyle='--', alpha=0.7, label='Missing Period' if ts == missing_timestamps[0] else "")

    # Highlight unexpected timestamps
    for ts in extra_timestamps:
        plt.axvline(ts, color='orange', linestyle='--', alpha=0.7, label='Unexpected Period' if ts == extra_timestamps[0] else "")

    # Add legend and labels
    plt.title("Data Plot with Highlighted Broken Periodicity")
    plt.xlabel("Time")
    plt.ylabel("Values")
    plt.legend()
    plt.tight_layout()

    # Show the plot
    plt.show()


def handle_nans_with_interpolation(df: pd.DataFrame, name:str) -> pd.DataFrame:
    """
    Checks each column of the DataFrame for NaNs. If a column has more than 3 consecutive NaNs,
    it raises a ValueError. Otherwise, it fills the NaNs using bi-directional interpolation.

    Parameters:
        df (pd.DataFrame): DataFrame with time series data (indexed by pd.Timestamp).

    Returns:
        pd.DataFrame: DataFrame with NaNs filled using bi-directional interpolation.

    Raises:
        ValueError: If any column contains more than 3 consecutive NaNs.
    """
    def check_consecutive_nans(series: pd.Series):
        # Find consecutive NaNs
        consecutive_nans = series.isna().astype(int).groupby(series.notna().cumsum()).cumsum()
        # Check if any sequence of NaNs exceeds 3
        if consecutive_nans.max() > 3:
            raise ValueError(f"Column '{series.name}' in {name} contains more than 3 consecutive NaNs.")

    df_copy = df.copy()
    for column in df_copy.columns:
        # Check if the column has more than 3 consecutive NaNs
        check_consecutive_nans(df_copy[column])
        # Fill NaNs using bi-directional interpolation
        df_copy[column] = df_copy[column].interpolate(method='linear', limit_direction='both')

    return df_copy
def fix_broken_periodicity_with_interpolation(df:pd.DataFrame, name:str) -> pd.DataFrame:
    """
    Fixes broken hourly periodicity by adding missing timestamps if fewer than 3 consecutive points are missing.
    Raises an error if more than 3 consecutive timestamps are missing.
    Missing values are filled using interpolation.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame with a datetime index.

    Returns:
        pd.DataFrame: A DataFrame with missing points added and values interpolated where necessary.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError(f"The DataFrame {name} must have a datetime index.")

    # Generate the full expected hourly range
    expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')

    # Find missing timestamps
    missing_timestamps = expected_index.difference(df.index)

    if missing_timestamps.empty:
        print(f"The DataFrame {name} is already hourly with no missing segments.")
        return df

    # Identify consecutive missing groups
    missing_diffs = missing_timestamps.to_series().diff().dt.seconds
    consecutive_missing_groups = missing_diffs != 3600  # Check if time diff isn't exactly 1 hour

    # Label consecutive groups
    missing_timestamps_df = pd.DataFrame({'timestamp': missing_timestamps})
    missing_timestamps_df['group'] = consecutive_missing_groups.cumsum()

    # Check for groups with more than 3 missing points
    for group_id, group in missing_timestamps_df.groupby('group'):
        if len(group) > 3:
            raise ValueError(f"More than 3 consecutive missing timestamps detected: {group['timestamp'].values} in {name}")

    # Add missing timestamps back to the DataFrame
    fixed_df = df.reindex(df.index.union(missing_timestamps).sort_values())

    # Interpolate to fill missing values
    fixed_df = fixed_df.interpolate(method='time')

    print(f"Added and interpolated {len(missing_timestamps)} missing timestamps in {name}.")

    return fixed_df
def validate_dataframe(df, name:str=''):
    ''' check for nans, missing values and preiodicity in a time-series dataframe '''
    # Check for NaNs
    has_nans = df.isnull().any().any()
    if has_nans:
        print(f"ERROR! {name} DataFrame contains NaN values.")
        df = handle_nans_with_interpolation(df, name)

    # Check if the index is in ascending order
    index_ascending = df.index.is_monotonic_increasing
    if not index_ascending:
        print(f"ERROR! {name} The index is not in ascending order.")
        raise ValueError("Data is not in ascending order.")

    # Check if the index is hourly with no missing segments
    is_hourly = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h').equals(df.index)
    if not is_hourly:
        print(f"ERROR! {name} The data is not hourly or has missing segments.")
        df = fix_broken_periodicity_with_interpolation(df, name)
    return df

# merger with SMRD target column
target = 'wind_offshore'
df_om_prep.dropna(inplace=True, how='any')
df_hist = pd.merge(df_om_prep, df_smard[target], left_index=True, right_index=True, how="inner")
df_forecast:pd.DataFrame = df_om_prep[cutoff+timedelta(hours=1) : cutoff+timedelta(hours=1 + 7*24)]
df_hist:pd.DataFrame = validate_dataframe(df_hist, 'df_hist')
df_forecast = validate_dataframe(df_forecast, 'df_forecast')
df_hist = df_hist[df_hist.index[-1]-pd.Timedelta(weeks=50):]
print(f"Features {len(df_hist.columns)-1} hist.shape={df_hist.shape} ({int(len(df_hist) / 7 / 24)} weeks) forecast.shape={df_forecast.shape}")


In [ ]:
from forecasting_modules import compute_timeseries_split_cutoffs, compute_error_metrics
cutoffs = compute_timeseries_split_cutoffs(
    df_hist.index,
    horizon=len(df_forecast.index),
    delta=len(df_forecast.index),
    folds=5,
    min_train_size=30*24
)
dfs = []
results = {}
metrics = []
for i, cutoff in enumerate(cutoffs):
    print(f"Processing {i}/{len(cutoffs)}")
    mask = (df_hist.index > cutoff) & (df_hist.index <= cutoff + pd.Timedelta(hours=len(df_forecast.index)))
    mask_ = (df_smard.index > cutoff) & (df_smard.index <= cutoff + pd.Timedelta(hours=len(df_forecast.index)))
    actual = df_hist[target][mask]
    predicted = df_smard[f"{target}_forecasted"][mask_]
    df = pd.DataFrame({
        f'{target}_actual':actual.values,
        f'{target}_fitted': predicted.values,
        f'{target}_lower': np.zeros_like(actual.values),
        f'{target}_upper': np.zeros_like(actual.values)
    }, index=actual.index)
    dfs.append(copy.deepcopy(df))
    metrics.append( compute_error_metrics(target, df) )

ave_metrics = {
    key: np.mean( [metrics[i][key] for i in range(len((metrics)))] ) for key in list(metrics[0].keys())
}
metrics.append(ave_metrics)
print(len(metrics))

In [ ]:
from data_modules import plot_time_series_with_residuals
n = 3
forecast = copy.deepcopy(dfs[-1])
forecast[:] = 0
tasks = [
    {'model':'SMARD','n':n,'name':'SMARD','lw':0.7,'color':'red','ci_alpha':0.0,
     'results':dfs[-n:],'metrics':metrics[-n-1:],'forecast':forecast}
]
plot_time_series_with_residuals(
    tasks=tasks,
    target=target,
    ylabel='Off-shore Wind Generation'
)

In [ ]:
# Preprocess SMARD data
def aggregate_smard_data(df_smard, countries:list):
    # 1. Creating the 'other_gen' column
    df_smard['other_gen'] = df_smard[['biomass', 'hydropower', 'lignite', 'hard_coal', 'natural_gas',
                                      'pumped_storage', 'other_conventional', 'other_renewables']].sum(axis=1)

    # 2. Creating 'net_import' and 'net_export' columns
    import_columns = [f"{country}_import" for country in countries]
    export_columns = [f"{country}_export" for country in countries]

    df_smard['net_import'] = df_smard[import_columns].sum(axis=1)
    df_smard['net_export'] = df_smard[export_columns].sum(axis=1)

    # 3. Selecting the required columns for the new dataframe
    selected_columns = ['other_gen', 'net_import', 'net_export', 'total_grid_load',
                        'residual_load', 'solar', 'wind_offshore', 'wind_onshore']

    df_new = df_smard[selected_columns]
df_smard_ = aggregate_smard_data(df_smard, list(DataEnergySMARD.country_map.keys()))

In [ ]:
def prepare_data_for_offshore_wind_gen(df_om, df_om_f):
    df = df
    location_suffix="_hsee"
    # 1. Filter for relevant location
    cols_to_keep = [c for c in df.columns if c.endswith(location_suffix)]
    df = df[cols_to_keep].copy()

In [ ]:
def get_df_hist_df_forecast(verbose:bool):
    data_dir = '../database/'
    # df = pd.read_parquet(data_dir + 'latest.parquet')
    df_smard = pd.read_parquet(data_dir + 'smard/' + 'history.parquet')

    def aggregate_smard_data(df_smard, countries:list):
        # 1. Creating the 'other_gen' column
        df_smard['other_gen'] = df_smard[['biomass', 'hydropower', 'lignite', 'hard_coal', 'natural_gas',
                                          'pumped_storage', 'other_conventional', 'other_renewables']].sum(axis=1)

        # 2. Creating 'net_import' and 'net_export' columns
        import_columns = [f"{country}_import" for country in countries]
        export_columns = [f"{country}_export" for country in countries]

        df_smard['net_import'] = df_smard[import_columns].sum(axis=1)
        df_smard['net_export'] = df_smard[export_columns].sum(axis=1)

        # 3. Selecting the required columns for the new dataframe
        selected_columns = ['other_gen', 'net_import', 'net_export', 'total_grid_load',
                            'residual_load', 'solar', 'wind_offshore', 'wind_onshore']

        df_new = df_smard[selected_columns]
    df_smard_ = aggregate_smard_data(df_smard, list(DataEnergySMARD.country_map.keys()))

    def aggregate_openmeteo_data(df_smard, locations_:list):



    df_om = pd.read_parquet(data_dir + 'openmeteo/' + 'history.parquet')
    df_om_f = pd.read_parquet(data_dir + 'openmeteo/' + 'forecast.parquet')
    df_es = pd.read_parquet(data_dir + 'epexspot/' + 'history.parquet')



    if verbose:
        print(f"SMARD data shapes hist={df_smard.shape} start={df_smard.index[0]} end={df_smard.index[-1]}")
        print(f"Openmeteo data shapes hist={df_om.shape} start={df_om.index[0]} end={df_om.index[-1]}")
        print(f"Openmeteo data shapes forecast={df_om_f.shape} start={df_om_f.index[0]} end={df_om_f.index[-1]}")
        print(f"EPEXSPOT data shapes hist={df_es.shape} start={df_es.index[0]} end={df_es.index[-1]}")

    df_hist = pd.merge(df_smard, df_om, left_index=True, right_index=True)
    df_hist = pd.merge(df_hist, df_es, left_index=True, right_index=True)

    if verbose:
        print(f"Merged hist dataframes shape")



In [ ]:
df_smard.tail(n=5)

In [ ]:
def plot_gen_load_with_forecasts(df_smard_):
    
    df_smard_ = df_smard_[~(df_smard_[['total_grid_load','residual_load','solar','wind_offshore','wind_onshore']] == 0).all(axis=1)]
    
    fig,axes = plt.subplots(figsize=(10,10), nrows=6, sharex='all')
    ax = axes[0]
    
    ax.plot(df_smard_.index,df_smard_['total_grid_load'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['total_grid_load_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Total Load')
    
    ax = axes[1]
    ax.plot(df_smard_.index,df_smard_['residual_load'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['residual_load_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Residual Load')
    
    ax = axes[2]
    ax.plot(df_smard_.index,df_smard_['solar'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['solar_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Solar Generation')
    
    ax = axes[3]
    ax.plot(df_smard_.index,df_smard_['wind_offshore'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['wind_offshore_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Wind Off-shore')
    
    ax = axes[4]
    ax.plot(df_smard_.index,df_smard_['wind_onshore'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['wind_onshore_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Wind On-shore')
    
    ax = axes[5]
    ax.plot(df_smard_.index,df_smard_[['biomass','hydropower','lignite','hard_coal','natural_gas','pumped_storage','other_conventional','other_renewables']].aggregate(func='sum',axis=1), color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['other_gen_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Other Generation')
    
    plt.tight_layout()
    plt.show()
plot_gen_load_with_forecasts(df_smard.tail(30*24))

In [ ]:
from data_collection_modules import DataEnergySMARD
def plot_cross_border_trade_smard(df_smard_, by_country:bool):
    if by_country:
        fig,axes = plt.subplots(figsize=(10,12), nrows=len(list(DataEnergySMARD.country_map.keys()))+1, sharex='all')
    else: 
        fig,ax = plt.subplots(figsize=(10,6), nrows=1)
        axes = [ax]
    if by_country:
        for ax, country in zip(axes, list(DataEnergySMARD.country_map.keys())):
            import_ = df_smard_[f"{country}_import"]
            export_ = df_smard_[f"{country}_export"]
            # ax.plot(df_smard_.index,import_, color='blue', label='Import')
            # ax.plot(df_smard_.index,export_, color='red', label='Export')
            ax.plot(df_smard_.index,import_ + export_, color='black', label='Trade')
            ax.set_ylabel(country)
            ax.axhline(0,color='gray',linestyle=':')

    net_import = df_smard_[[f"{country}_import" for country in list(DataEnergySMARD.country_map.keys())]].aggregate(func='sum',axis=1)
    net_export = df_smard_[[f"{country}_export" for country in list(DataEnergySMARD.country_map.keys())]].aggregate(func='sum',axis=1)
    
    # Calculate moving averages
    window_size = 24  # Adjust the window size as needed
    net_import_ma = net_import.rolling(window=window_size).mean()
    net_export_ma = net_export.rolling(window=window_size).mean()
    net_sum_ma = (net_import + net_export).rolling(window=window_size).mean()
    
    # Plot the original data
    axes[-1].plot(df_smard_.index, net_import, color='blue', label='Import')
    axes[-1].plot(df_smard_.index, net_export, color='red', label='Export')
    axes[-1].plot(df_smard_.index, net_export + net_import, color='black', label='Sum')
    
    # Plot the moving averages
    axes[-1].plot(df_smard_.index, net_import_ma, color='blue', linestyle='--', label='Import (MA)')
    axes[-1].plot(df_smard_.index, net_export_ma, color='red', linestyle='--', label='Export (MA)')
    axes[-1].plot(df_smard_.index, net_sum_ma, color='black', linestyle='--', label='Sum (MA)')
    
    # Add reference line and legend
    axes[-1].axhline(0, color='gray', linestyle=':')
    axes[-1].legend(loc='upper right')
    
    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()
plot_cross_border_trade_smard(df_smard.tail(n=30*24), by_country=False)

# OPENMETEO

In [ ]:
from data_collection_modules import locations, OpenMeteo
def plot_openmeteo(df, df_f, suffix):

    fig, axes = plt.subplots(nrows=len(list(OpenMeteo.variables_standard)), sharex='all', figsize=(10,12))
    for ax, quantity in zip(axes, list(OpenMeteo.variables_standard)):
        ax.plot(df.index, df[f"{quantity}{suffix}"], color='black', ls='-', label='Historic')
        ax.plot(df_f.index, df_f[f"{quantity}{suffix}"], color='black', ls='--', label='Forecast')
        ax.set_ylabel(quantity)

        ave = df[[f"{quantity}{loc['suffix']}" for loc in locations]].aggregate(func='mean',axis=1)
        ave_f = df[[f"{quantity}{loc['suffix']}" for loc in locations]].aggregate(func='mean',axis=1)

        ax.plot(ave.index, ave, color='gray', ls='-', label='Historic')
        ax.plot(ave_f.index, ave_f, color='gray', ls='--', label='Forecast')
        ax.set_ylabel(quantity)
    plt.tight_layout()
    plt.show()
plot_openmeteo(df=df_om.tail(n=30*24),df_f=df_om_f,suffix='_hsee')
    

In [ ]:
def visualize_energy_data(df):
    # Ensure datetime index
    df_month = df.tail(30*24)

    # List of countries
    countries = ['austria', 'belgium', 'czechia', 'denmark', 'france', 'luxembourg',
                 'netherlands', 'norway', 'poland', 'sweden', 'switzerland']

    # Sum '_export' and '_import' columns for each country
    df_flows = pd.DataFrame()
    for country in countries:
        df_flows[country] = df_month[f'{country}_export'] + df_month[f'{country}_import']

    # Remaining columns
    remaining_columns = ['other_gen', 'residual_load_forecast', 'solar', 'total_gen',
                         'total_grid_load', 'wind_offshore', 'wind_onshore']
    df_remaining = df_month[remaining_columns]


    # Plotting
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

    # First panel: stacked bar chart of df_flows
    df_flows.plot(kind='bar', stacked=True, ax=axes[0], width=1.0, linewidth=0)

    axes[0].set_title('Hourly Energy Flows by Country')
    axes[0].set_ylabel('Total Energy Flow')
    axes[0].legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    axes[0].set_xlabel('')

    # Adjust x-axis ticks on the bar chart
    num_ticks = len(df_flows)
    tick_positions = range(0, num_ticks, 24)  # Show a tick every 24 bars (once per day)
    tick_labels = df_flows.index[::24].strftime('%Y-%m-%d')
    axes[0].set_xticks(tick_positions)
    axes[0].set_xticklabels(tick_labels, rotation=45, ha='right')

    # Second panel: lines of remaining columns
    df_remaining.plot(ax=axes[1])
    axes[1].set_title('Hourly Other Energy Data')
    axes[1].set_ylabel('Value')
    axes[1].legend(loc='upper left', bbox_to_anchor=(1.02, 1))

    # Adjust x-axis ticks on the second panel
    axes[1].xaxis.set_major_locator(mdates.DayLocator(interval=3))
    axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
    axes[1].set_xlabel('Date')

    plt.tight_layout()
    plt.show()
visualize_energy_data(df_smard)

E

# Combined Weather and Energy

In [ ]:
def plot_smard_openmeteo(df_smard, df_om, df_om_f, om_suffix):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 10),sharex='all')
    
    ax = axes[0]
    ax_ = ax.twinx()
    ax.plot(df_smard.index, df_smard['solar'], color='black', label='Solar Energy Generation')
    ax.set_ylabel('Solar Energy Generation')
    ax_.plot(df_om.index, df_om[f'shortwave_radiation{om_suffix}'], color='red', label='Historic')
    ax_.plot(df_om_f.index, df_om_f[f'shortwave_radiation{om_suffix}'], color='red', ls='--', label='Forecast')
    ax_.set_ylabel('Shortwave Radiation')

    ax = axes[1]
    ax_ = ax.twinx()
    ax.plot(df_smard.index, df_smard['wind_onshore'], color='black', label='Solar Energy Generation')
    ax.set_ylabel('On-shore Wind Generation')
    ax_.plot(df_om.index, df_om[f'wind_speed_10m{om_suffix}'], color='red', label='Historic')
    ax_.plot(df_om_f.index, df_om_f[f'wind_speed_10m{om_suffix}'], color='red', ls='--', label='Forecast')
    ax.axvline(df_om_f.index[0])
    ax_.set_ylabel('Wind Speed')
    
    plt.tight_layout()
    plt.show()
plot_smard_openmeteo(df_smard.tail(30*24), df_om.tail(30*24), df_om_f, om_suffix='_hsee')
    

In [ ]:
df_es.tail(24*30).plot(figsize=(16,5))

In [ ]:
df_es.tail()

In [ ]:
def visualize_time_series(df, df_f, suffix, figsize=(15, 20)):
    """
    Visualizes each column with a specified suffix in separate panels with a shared X-axis.

    Parameters:
    - df (pd.DataFrame): DataFrame with time-series data indexed by pd.Timestamp.
    - suffix (str): Suffix to filter the columns of interest.
    - figsize (tuple): Size of the figure for the plots.
    """
    # Filter columns based on the suffix
    filtered_columns = [col for col in df.columns if col.endswith(suffix)]

    if not filtered_columns:
        raise ValueError(f"No columns found with suffix '{suffix}' in the DataFrame.")

    num_columns = len(filtered_columns)

    # Create a shared X-axis for all subplots
    fig, axes = plt.subplots(num_columns, 1, figsize=figsize, sharex=True)
    if num_columns == 1:
        axes = [axes]  # Ensure axes is always iterable

    for ax, column in zip(axes, filtered_columns):
        ax.plot(df.index, df[column], label=column)
        ax.plot(df_f.index, df_f[column], ls='--')
        ax.set_ylabel(column.replace(suffix, '').replace('_', ' ').capitalize())
        ax.legend(loc='upper right')
        ax.grid(True)

    # Set shared X-axis label
    plt.xlabel("Time")
    plt.tight_layout()
    plt.show()
visualize_time_series(df=df_om,df_f = df_om_f, suffix='hsee')

def

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_multivariate_timeseries(df, df_fore):
    # Create a subplot with 3 rows
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

    # Plot 'DA_auction_price' on the first subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['DA_auction_price'], name='DA Auction Price'),
                  row=1, col=1)

    # Plot 'total_gen', 'total_grid_load', 'residual_load_forecast' on the second subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['total_gen'], name='Total Generation'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['total_grid_load'], name='Total Grid Load'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['residual_load_forecast'], name='Residual Load Forecast'), row=2, col=1)

    # Plot 'temperature_2m_hsee' and 'temperature_2m_solw' on the third subplot
    for df_,ls in zip([df, df_fore],['solid', 'dash']):
        fig.add_trace(go.Scatter(
            x=df_.index,
            y=df_['temperature_2m_hsee'],
            name='Temperature 2m HSEE',
            line=dict(dash=ls,color='blue')
        ), row=3, col=1)

        fig.add_trace(go.Scatter(
            x=df_.index,
            y=df_['temperature_2m_solw'],
            name='Temperature 2m SOLW',
            line=dict(dash=ls,color='green')
        ), row=3, col=1)
    # Add a vertical line for today across all subplots
    # today_line = dict(type='line', x0=pd.Timestamp.today(), y0=0, x1=pd.Timestamp.today(),
    #                   y1=1, xref='x', yref='paper', line=dict(color='black', width=2, dash='dash'))
    # fig.add_shape(today_line, row='all', col=1)
    fig.add_vline(x=pd.Timestamp.today(), line_width=1, line_dash="dash", line_color="gray")

    # Update layout
    fig.update_layout(height=900, title='Multivariate Time Series Analysis',
                      xaxis_title='DateTime', yaxis_title='Values')

    # Show the figure
    fig.show()
plot_multivariate_timeseries(df=df_hist.tail(1000), df_fore = df_fore)

# Plot SMARD data

In [ ]:
from data_modules.collect_data_smard import DataEnergySMARD
countries = list(DataEnergySMARD.country_map.keys())
# Create a subplot with 3 rows
fig = make_subplots(rows=len(countries), cols=1, shared_xaxes=True, vertical_spacing=0.02)

df_hist_ = df_hist.tail(4*7*24)
for i, country in enumerate(countries):
    fig.add_trace(go.Scatter(
        x=df_hist_.index,
        y=df_hist_[f'{country}_export'],
        name=f'{country}',
        line=dict(color='blue')  # Set the line color here
    ), row=i+1, col=1)
    fig.add_trace(go.Scatter(
        x=df_hist_.index,
        y=df_hist_[f'{country}_import'],
        name=None,#f'{country}',
        line=dict(color='red')  # Set the line color here
    ), row=i+1, col=1)
# Update layout
fig.update_layout(height=1800, title='International trade',
xaxis_title='DateTime', yaxis_title='Values')

# Show the figure
fig.show()